# Serie 2
## Esercizio 1

In [1]:
class Autostrada {
        public int entrate = 0;
        public int uscite = 0;
        public int pedaggi = 0;
}

class Automobilista implements Runnable {
        private final int id;
        private final Autostrada autostrada;
        private final int delay;
        private int pedaggiPagati = 0;

        public Automobilista(int id, Autostrada state, int delay) {
                this.autostrada = state;
                this.delay = delay;
                this.id = id;
                this.pedaggiPagati = 0;
                System.out.println("Automobilista " + id + ": creato con " + delay
                                + " ms di percorrenza");
        }

        public int getPedaggiPagati() {
                return pedaggiPagati;
        }

        public int getID() {
                return id;
        }

        @Override
        public void run() {
                System.out.println("Automobilista " + id + ": partito");

                for (int i = 0; i < 500; i++) {
                        vaiVersoAutostrada();

                        autostrada.entrate++;

                        int pedaggioTratta = percorriAutostrada();

                        autostrada.uscite++;
                        autostrada.pedaggi += pedaggioTratta;
                        pedaggiPagati += pedaggioTratta;
                }
                System.out.println("Automobilista " + id + ": terminato");
        }

        private void vaiVersoAutostrada() {
                try {
                        Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
                } catch (InterruptedException e) {
                        e.printStackTrace();
                }
        }

        private int percorriAutostrada() {
                try {
                        Thread.sleep(delay);
                } catch (InterruptedException e) {
                        e.printStackTrace();
                }
                return ThreadLocalRandom.current().nextInt(10, 20);
        }
}

In [2]:
final Collection<Automobilista> workers = new ArrayList<Automobilista>();
final Collection<Thread> threads = new ArrayList<Thread>();
final Random rand = new Random();

final Autostrada autostrada = new Autostrada();

for (int i = 0; i < 10; i++) {
        final int delay = 1 + rand.nextInt(5);
        final Automobilista a = new Automobilista(i, autostrada, delay);
        workers.add(a);
        threads.add(new Thread(a));
}

System.out.println("Simulation started");
System.out.println("------------------------------------");
threads.forEach(Thread::start);

try {
        for (final Thread t : threads){
                t.join();
        }
} catch (final InterruptedException e) {
        System.exit(1);
}

System.out.println("------------------------------------");
System.out.println("Simulation finished");

int totalePedaggiUtenti = 0;
for (final Automobilista a : workers) {
        int pedaggiPagati = a.getPedaggiPagati();
        totalePedaggiUtenti += pedaggiPagati;
        System.out.println("Automobilista " + a.getID() + " ha pagato " + pedaggiPagati);
}

System.out.println("Automobilisti totale pedaggi: " + totalePedaggiUtenti);
System.out.println("Autostrada totale pedaggi   : " + autostrada.pedaggi);
System.out.println("Autostrada totale entrate :" + autostrada.entrate);
System.out.println("Autostrada totale uscite  :" + autostrada.uscite);

Automobilista 0: creato con 5 ms di percorrenza
Automobilista 1: creato con 2 ms di percorrenza
Automobilista 2: creato con 5 ms di percorrenza
Automobilista 3: creato con 4 ms di percorrenza
Automobilista 4: creato con 1 ms di percorrenza
Automobilista 5: creato con 3 ms di percorrenza
Automobilista 6: creato con 1 ms di percorrenza
Automobilista 7: creato con 3 ms di percorrenza
Automobilista 8: creato con 1 ms di percorrenza
Automobilista 9: creato con 1 ms di percorrenza
Simulation started
------------------------------------
Automobilista 0: partito
Automobilista 1: partito
Automobilista 2: partito
Automobilista 4: partito
Automobilista 5: partito
Automobilista 7: partito
Automobilista 3: partito
Automobilista 8: partito
Automobilista 6: partito
Automobilista 9: partito
Automobilista 4: terminato
Automobilista 8: terminato
Automobilista 6: terminato
Automobilista 9: terminato
Automobilista 1: terminato
Automobilista 7: terminato
Automobilista 5: terminato
Automobilista 3: terminat

### Problematiche
Nella versione appena vista (versione originale), possiamo chiaramente notare come il numero di "Automobilisti totale pedaggi" risulta differente da quello di "Autostrada totale pedaggi". Possiamo inoltre notare che il numero di entrate non è concorde con il numero di uscite.  
  
Questo problema si verifica in quanto più Thread modificano dei valori mutabili dello stesso oggetto di classe Autostrada. Capita così una cosidetta "race condition" che causa risultati simili a quelli dimostrati precedentemente.


### Risoluzione
Per sistemare il problema abbiamo tre possibilità:
- Rendere i metodi che fanno uso di `this.autostrada` `synchronized`
- Utilizzare dei synchronized blocks quando eseguiamo delle operazioni su `this.autostrada`
- Fare uso di lock espliciti

#### Synchronized methods
Per sistemare il problema utilizzando i `synchronized` methods possiamo sistemare `addPedaggio()`, `addEntrata()` e `addUscita()` in modo da essere synchronized, in quanto questi eseguono delle operazioni (che devono essere atomiche) su un istanza di `Autostrada` che è condivisa con tutti le istanze di `Automobilista`.

In [3]:
class Autostrada {
	private int entrate = 0;
	private int uscite = 0;
	private int pedaggi = 0;

	synchronized void addPedaggio(int pedaggio){
		pedaggi+= pedaggio;
	}

	synchronized void addEntrata(){
		entrate++;
	}

	synchronized void addUscita(){
		uscite++;
	}

	public int getPedaggi() {
		return pedaggi;
	}

	public int getEntrate() {
		return entrate;
	}

	public int getUscite() {
		return uscite;
	}
}

class Automobilista implements Runnable {
	private final int id;
	private final Autostrada autostrada;
	private final int delay;
	private int pedaggiPagati = 0;

	public Automobilista(int id, Autostrada state, int delay) {
		this.autostrada = state;
		this.delay = delay;
		this.id = id;
		this.pedaggiPagati = 0;
		System.out.println("Automobilista " + id + ": creato con " + delay
				+ " ms di percorrenza");
	}

	public int getPedaggiPagati() {
		return pedaggiPagati;
	}

	public int getID() {
		return id;
	}

	@Override
	public void run() {
		System.out.println("Automobilista " + id + ": partito");

		for (int i = 0; i < 500; i++) {
			vaiVersoAutostrada();
			
			autostrada.addEntrata();
			
			int pedaggioTratta = percorriAutostrada();

			autostrada.addUscita();
			autostrada.addPedaggio(pedaggioTratta);
			pedaggiPagati += pedaggioTratta;
		}
		System.out.println("Automobilista " + id + ": terminato");
	}

	private void vaiVersoAutostrada() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	private int percorriAutostrada() {
		try {
			Thread.sleep(delay);
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
		return ThreadLocalRandom.current().nextInt(10, 20);
	}
}

In [4]:
final Collection<Automobilista> workers = new ArrayList<Automobilista>();
final Collection<Thread> threads = new ArrayList<Thread>();
final Random rand = new Random();

final Autostrada autostrada = new Autostrada();

for (int i = 0; i < 10; i++) {
    final int delay = 1 + rand.nextInt(5);
    final Automobilista a = new Automobilista(i, autostrada, delay);
    workers.add(a);
    threads.add(new Thread(a));
}

System.out.println("Simulation started");
System.out.println("------------------------------------");
threads.forEach(Thread::start);
try {
    for (final Thread t : threads){
        t.join();
    }
} catch (final InterruptedException e) {
    System.exit(1);
}

System.out.println("------------------------------------");
System.out.println("Simulation finished");

int totalePedaggiUtenti = 0;
for (final Automobilista a : workers) {
    int pedaggiPagati = a.getPedaggiPagati();
    totalePedaggiUtenti += pedaggiPagati;
    System.out.println("Automobilista " + a.getID() + " ha pagato " + pedaggiPagati);
}

System.out.println("Automobilisti totale pedaggi: " + totalePedaggiUtenti);
System.out.println("Autostrada totale pedaggi   : " + autostrada.getPedaggi());
System.out.println("Autostrada totale entrate :" + autostrada.getEntrate());
System.out.println("Autostrada totale uscite  :" + autostrada.getUscite());

Automobilista 0: creato con 4 ms di percorrenza
Automobilista 1: creato con 3 ms di percorrenza
Automobilista 2: creato con 5 ms di percorrenza
Automobilista 3: creato con 5 ms di percorrenza
Automobilista 4: creato con 4 ms di percorrenza
Automobilista 5: creato con 2 ms di percorrenza
Automobilista 6: creato con 5 ms di percorrenza
Automobilista 7: creato con 1 ms di percorrenza
Automobilista 8: creato con 1 ms di percorrenza
Automobilista 9: creato con 4 ms di percorrenza
Simulation started
------------------------------------
Automobilista 0: partito
Automobilista 3: partito
Automobilista 8: partito
Automobilista 2: partito
Automobilista 9: partito
Automobilista 4: partito
Automobilista 7: partito
Automobilista 6: partito
Automobilista 1: partito
Automobilista 5: partito
Automobilista 8: terminato
Automobilista 7: terminato
Automobilista 5: terminato
Automobilista 1: terminato
Automobilista 9: terminato
Automobilista 4: terminato
Automobilista 0: terminato
Automobilista 3: terminat

Il risultato è quindi evidente: il totale dei pedaggi è concorde fra gli automobilisti e l'autostrada, mentre il totale delle entrate e delle uscite è il medesimo. Il programma è ora *thread safe*.

#### Synchronized blocks
Un altro metodo per la risoluzione del problema è l'utilizzo dei `synchronized` blocks. I synchronized blocks permettono di rendere un operazione atomica: in questo modo l'operazione ha completo controllo sulle variabili che utilizza, in modo da evitare race conditions.

In [8]:
class Autostrada {
	public int entrate = 0;
	public int uscite = 0;
	public int pedaggi = 0;
}

class Automobilista implements Runnable {
	private final int id;
	private final Autostrada autostrada;
	private final int delay;
	private int pedaggiPagati = 0;

	public Automobilista(int id, Autostrada state, int delay) {
		this.autostrada = state;
		this.delay = delay;
		this.id = id;
		this.pedaggiPagati = 0;
		System.out.println("Automobilista " + id + ": creato con " + delay
				+ " ms di percorrenza");
	}

	public int getPedaggiPagati() {
		return pedaggiPagati;
	}

	public int getID() {
		return id;
	}

	@Override
	public void run() {
		System.out.println("Automobilista " + id + ": partito");

		for (int i = 0; i < 500; i++) {
			vaiVersoAutostrada();

			synchronized (autostrada) {
				autostrada.entrate++;
			}

			int pedaggioTratta = percorriAutostrada();

			synchronized (autostrada){
				autostrada.uscite++;
				autostrada.pedaggi += pedaggioTratta;
				pedaggiPagati += pedaggioTratta;
			}
		}
		System.out.println("Automobilista " + id + ": terminato");
	}

	private void vaiVersoAutostrada() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	private int percorriAutostrada() {
		try {
			Thread.sleep(delay);
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
		return ThreadLocalRandom.current().nextInt(10, 20);
	}
}

In [10]:
final Collection<Automobilista> workers = new ArrayList<Automobilista>();
final Collection<Thread> threads = new ArrayList<Thread>();
final Random rand = new Random();
final Autostrada autostrada = new Autostrada();

for (int i = 0; i < 10; i++) {
    final int delay = 1 + rand.nextInt(5);
    final Automobilista a = new Automobilista(i, autostrada, delay);
    workers.add(a);
    threads.add(new Thread(a));
}

System.out.println("Simulation started");
System.out.println("------------------------------------");
threads.forEach(Thread::start);

try {
    for (final Thread t : threads)
        t.join();
} catch (final InterruptedException e) {
    System.exit(1);
}

System.out.println("------------------------------------");
System.out.println("Simulation finished");

int totalePedaggiUtenti = 0;
for (final Automobilista a : workers) {
    int pedaggiPagati = a.getPedaggiPagati();
    totalePedaggiUtenti += pedaggiPagati;
    System.out.println("Automobilista " + a.getID() + " ha pagato " + pedaggiPagati);
}

System.out.println("Automobilisti totale pedaggi: " + totalePedaggiUtenti);
System.out.println("Autostrada totale pedaggi   : " + autostrada.pedaggi);
System.out.println("Autostrada totale entrate :" + autostrada.entrate);
System.out.println("Autostrada totale uscite  :" + autostrada.uscite);

Automobilista 0: creato con 5 ms di percorrenza
Automobilista 1: creato con 2 ms di percorrenza
Automobilista 2: creato con 3 ms di percorrenza
Automobilista 3: creato con 1 ms di percorrenza
Automobilista 4: creato con 4 ms di percorrenza
Automobilista 5: creato con 3 ms di percorrenza
Automobilista 6: creato con 3 ms di percorrenza
Automobilista 7: creato con 4 ms di percorrenza
Automobilista 8: creato con 2 ms di percorrenza
Automobilista 9: creato con 3 ms di percorrenza
Simulation started
------------------------------------
Automobilista 0: partito
Automobilista 1: partito
Automobilista 2: partito
Automobilista 3: partito
Automobilista 4: partito
Automobilista 5: partito
Automobilista 6: partito
Automobilista 7: partito
Automobilista 8: partito
Automobilista 9: partito
Automobilista 3: terminato
Automobilista 8: terminato
Automobilista 1: terminato
Automobilista 5: terminato
Automobilista 2: terminato
Automobilista 9: terminato
Automobilista 6: terminato
Automobilista 4: terminat

Il nostro programma è nuovamente *thread safe* grazie ai `synchronized` blocks.

#### Explicit Locks
Un altro metodo per rendere il programma thread safe è l'utilizzo di explicit locks.  
Nel nostro caso metteremo un explicit lock sull'oggetto `autostrada` in quanto questo è condiviso fra le varie istanze di Automobilista, ed è la causa delle nostre race conditions

In [12]:
import java.util.concurrent.ThreadLocalRandom;
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

class Autostrada {
    public int entrate = 0;
    public int uscite = 0;
    public int pedaggi = 0;

    public Lock lock = new ReentrantLock();
}

class Automobilista implements Runnable {
    private final int id;
    private final Autostrada autostrada;
    private final int delay;
    private int pedaggiPagati = 0;

    public Automobilista(int id, Autostrada state, int delay) {
        this.autostrada = state;
        this.delay = delay;
        this.id = id;
        this.pedaggiPagati = 0;
        System.out.println("Automobilista " + id + ": creato con " + delay
                + " ms di percorrenza");
    }

    public int getPedaggiPagati() {
        return pedaggiPagati;
    }

    public int getID() {
        return id;
    }

    @Override
    public void run() {
        System.out.println("Automobilista " + id + ": partito");

        for (int i = 0; i < 500; i++) {
            vaiVersoAutostrada();

            autostrada.lock.lock();
            autostrada.entrate++;
            autostrada.lock.unlock();


            int pedaggioTratta = percorriAutostrada();

            autostrada.lock.lock();
            autostrada.uscite++;
            autostrada.pedaggi += pedaggioTratta;
            pedaggiPagati += pedaggioTratta;
            autostrada.lock.unlock();
        }
        System.out.println("Automobilista " + id + ": terminato");
    }

    private void vaiVersoAutostrada() {
        try {
            Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }

    private int percorriAutostrada() {
        try {
            Thread.sleep(delay);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        return ThreadLocalRandom.current().nextInt(10, 20);
    }
}

In [13]:
final Collection<Automobilista> workers = new ArrayList<Automobilista>();
final Collection<Thread> threads = new ArrayList<Thread>();
final Random rand = new Random();

final Autostrada autostrada = new Autostrada();

for (int i = 0; i < 10; i++) {
    // Genera numero casuale tra 1 e 5 ms
    final int delay = 1 + rand.nextInt(5);
    final Automobilista a = new Automobilista(i, autostrada, delay);
    workers.add(a);
    threads.add(new Thread(a));
}

System.out.println("Simulation started");
System.out.println("------------------------------------");

threads.forEach(Thread::start);

try {
    for (final Thread t : threads)
        t.join();
} catch (final InterruptedException e) {
    System.exit(1);
}

System.out.println("------------------------------------");
System.out.println("Simulation finished");

int totalePedaggiUtenti = 0;
for (final Automobilista a : workers) {
    int pedaggiPagati = a.getPedaggiPagati();
    totalePedaggiUtenti += pedaggiPagati;
    System.out.println("Automobilista " + a.getID() + " ha pagato " + pedaggiPagati);
}

System.out.println("Automobilisti totale pedaggi: " + totalePedaggiUtenti);
System.out.println("Autostrada totale pedaggi   : " + autostrada.pedaggi);
System.out.println("Autostrada totale entrate :" + autostrada.entrate);
System.out.println("Autostrada totale uscite  :" + autostrada.uscite);

Automobilista 0: creato con 3 ms di percorrenza
Automobilista 1: creato con 5 ms di percorrenza
Automobilista 2: creato con 2 ms di percorrenza
Automobilista 3: creato con 1 ms di percorrenza
Automobilista 4: creato con 5 ms di percorrenza
Automobilista 5: creato con 3 ms di percorrenza
Automobilista 6: creato con 5 ms di percorrenza
Automobilista 7: creato con 4 ms di percorrenza
Automobilista 8: creato con 5 ms di percorrenza
Automobilista 9: creato con 4 ms di percorrenza
Simulation started
------------------------------------
Automobilista 3: partito
Automobilista 1: partito
Automobilista 2: partito
Automobilista 6: partito
Automobilista 9: partito
Automobilista 8: partito
Automobilista 7: partito
Automobilista 4: partito
Automobilista 5: partito
Automobilista 0: partito
Automobilista 3: terminato
Automobilista 2: terminato
Automobilista 0: terminato
Automobilista 5: terminato
Automobilista 9: terminato
Automobilista 7: terminato
Automobilista 4: terminato
Automobilista 1: terminat

Possiamo nuovamente notare come, anche con questo metodo, il nostro programma è thread safe.

## Esercizio 2

Anche in questo esercizio si verifica una Race Condition


In [15]:
class BagnoPubblico {
	private int totUtilizzi;
	private int totOccupati;
	private final int disponibili;
	private int occupati;

	public BagnoPubblico(int numeroBagno) {
		this.disponibili = numeroBagno;
		this.occupati = 0;
		this.totUtilizzi = 0;
		this.totOccupati = 0;
	}

	public boolean occupa() {
		// Verifica disponibilita bagni liberi!
		if (occupati < disponibili) {
			// Bagno libero! Occupa
			occupati++;
			totUtilizzi++;
		} else {
			// Tutti i bagni sono occupati!
			totOccupati++;
			return false;
		}

		// Utilizza il bagno
		utilizzaBagno();

		// Libera il bagno
		occupati--;
		return true;
	}

	private void utilizzaBagno() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(5, 15));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	public int getTotaleUtilizzo() {
		return totUtilizzi;
	}

	public int getTotaleOccupato() {
		return totOccupati;
	}
}

class Utente implements Runnable {
	private final int ID;
	private final BagnoPubblico bp;
	private int numUtilizzi;
	private int numOccupato;

	public Utente(BagnoPubblico bagno, int id) {
		this.bp = bagno;
		this.ID = id;
		this.numUtilizzi = 0;
		this.numOccupato = 0;
	}

	@Override
	public void run() {
		System.out.println(this + " inizio");
		for (int i = 0; i < 250; i++) {
			if (bp.occupa())
				numUtilizzi++;
			else
				numOccupato++;

			// Simula il tempo prima di dover tornare al bagno
			try {
				Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
			} catch (InterruptedException e) {
				e.printStackTrace();
			}
		}
		System.out.println(this + " termino");
	}

	public int getNumUtilizzi() {
		return numUtilizzi;
	}

	public int getNumOccupato() {
		return numOccupato;
	}

	@Override
	public String toString() {
		return "Utente" + ID;
	}
}

BagnoPubblico bp = new BagnoPubblico(2);

List<Thread> allPersons = new ArrayList<>();
List<Utente> allUsers = new ArrayList<>();
for (int i = 1; i <= 10; i++) {
    final Utente user = new Utente(bp, i);
    allUsers.add(user);
    allPersons.add(new Thread(user));
    System.out.println("Creato utente: " + user);
}

allPersons.forEach(Thread::start);

for (Thread thread : allPersons) {
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

int total = 0;
int totalOccupato = 0;
for (Utente utente : allUsers) {
    total += utente.getNumUtilizzi();
    totalOccupato += utente.getNumOccupato();
    System.out.println(
            utente + ": utilizzi: " + utente.getNumUtilizzi() + " occupato: " + utente.getNumOccupato());
}

System.out.println("Riepilogo utilizzi");
System.out.println("Totale utenti: " + total);
System.out.println("Totale Bagno: " + bp.getTotaleUtilizzo());

System.out.println("Riepilogo occupazione");
System.out.println("Totale utenti: " + totalOccupato);
System.out.println("Totale Bagno: " + bp.getTotaleOccupato());

Creato utente: Utente1
Creato utente: Utente2
Creato utente: Utente3
Creato utente: Utente4
Creato utente: Utente5
Creato utente: Utente6
Creato utente: Utente7
Creato utente: Utente8
Creato utente: Utente9
Creato utente: Utente10
Utente1 inizio
Utente7 inizio
Utente4 inizio
Utente2 inizio
Utente3 inizio
Utente6 inizio
Utente8 inizio
Utente9 inizio
Utente10 inizio
Utente5 inizio
Utente2 termino
Utente9 termino
Utente8 termino
Utente1 termino
Utente5 termino
Utente4 termino
Utente7 termino
Utente3 termino
Utente6 termino
Utente10 termino
Utente1: utilizzi: 46 occupato: 204
Utente2: utilizzi: 28 occupato: 222
Utente3: utilizzi: 80 occupato: 170
Utente4: utilizzi: 53 occupato: 197
Utente5: utilizzi: 50 occupato: 200
Utente6: utilizzi: 77 occupato: 173
Utente7: utilizzi: 69 occupato: 181
Utente8: utilizzi: 37 occupato: 213
Utente9: utilizzi: 23 occupato: 227
Utente10: utilizzi: 80 occupato: 170
Riepilogo utilizzi
Totale utenti: 543
Totale Bagno: 540
Riepilogo occupazione
Totale utenti: 195

Si noti come, anche in questo esercizio, si verifica una race condition che rende il "Totale utenti" differenti da "Totale bagno" sia per l'occupazione che per gli utilizzi.

### Problematiche
Il metodo `occupa()` di `BagnoPubblico` esegue delle operazioni che devono essere Thread safe, in quanto le istanze di `Utente` condividono la stessa istanza di `BagnoPubblico` e la utilizzano in più thread contemporaneamente.

### Risoluzione
Come prima, abbiamo a disposizione ancora tre possibilità per la risoluzione del problema. Vediamole quindi in dettaglio.

#### Synchronized Methods
Rendendo `occupa()` come `synchronized` possiamo risolvere la nostra race condition.

In [18]:
class BagnoPubblico {
	private int totUtilizzi;
	private int totOccupati;
	private final int disponibili;
	private int occupati;

	public BagnoPubblico(int numeroBagno) {
		this.disponibili = numeroBagno;
		this.occupati = 0;
		this.totUtilizzi = 0;
		this.totOccupati = 0;
	}

	synchronized public boolean occupa() {
		if (occupati < disponibili) {
			occupati++;
			totUtilizzi++;
		} else {
			totOccupati++;
			return false;
		}
		utilizzaBagno();
		occupati--;
		return true;
	}

	private void utilizzaBagno() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(5, 15));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	public int getTotaleUtilizzo() {
		return totUtilizzi;
	}

	public int getTotaleOccupato() {
		return totOccupati;
	}
}

In [17]:
class Utente implements Runnable {
	private final int ID;
	private final BagnoPubblico bp;
	private int numUtilizzi;
	private int numOccupato;

	public Utente(BagnoPubblico bagno, int id) {
		this.bp = bagno;
		this.ID = id;
		this.numUtilizzi = 0;
		this.numOccupato = 0;
	}

	@Override
	public void run() {
		System.out.println(this + " inizio");
		for (int i = 0; i < 250; i++) {
			if (bp.occupa()) {
				numUtilizzi++;
			}
			else {
				numOccupato++;
			}

			// Simula il tempo prima di dover tornare al bagno
			try {
				Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
			} catch (InterruptedException e) {
				e.printStackTrace();
			}
		}
		System.out.println(this + " termino");
	}

	public int getNumUtilizzi() {
		return numUtilizzi;
	}

	public int getNumOccupato() {
		return numOccupato;
	}

	@Override
	public String toString() {
		return "Utente" + ID;
	}
}

BagnoPubblico bp = new BagnoPubblico(2);
List<Thread> allPersons = new ArrayList<>();
List<Utente> allUsers = new ArrayList<>();
for (int i = 1; i <= 10; i++) {
    final Utente user = new Utente(bp, i);
    allUsers.add(user);
    allPersons.add(new Thread(user));
    System.out.println("Creato utente: " + user);
}

allPersons.forEach(Thread::start);

for (Thread thread : allPersons) {
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

int total = 0;
int totalOccupato = 0;
for (Utente utente : allUsers) {
    total += utente.getNumUtilizzi();
    totalOccupato += utente.getNumOccupato();
    System.out.println(
            utente + ": utilizzi: " + utente.getNumUtilizzi() + " occupato: " + utente.getNumOccupato());
}

System.out.println("Riepilogo utilizzi");
System.out.println("Totale utenti: " + total);
System.out.println("Totale Bagno: " + bp.getTotaleUtilizzo());

System.out.println("Riepilogo occupazione");
System.out.println("Totale utenti: " + totalOccupato);
System.out.println("Totale Bagno: " + bp.getTotaleOccupato());

Creato utente: Utente1
Creato utente: Utente2
Creato utente: Utente3
Creato utente: Utente4
Creato utente: Utente5
Creato utente: Utente6
Creato utente: Utente7
Creato utente: Utente8
Creato utente: Utente9
Creato utente: Utente10
Utente1 inizio
Utente10 inizio
Utente9 inizio
Utente8 inizio
Utente3 inizio
Utente2 inizio
Utente4 inizio
Utente5 inizio
Utente7 inizio
Utente6 inizio
Utente9 termino
Utente8 termino
Utente3 termino
Utente2 termino
Utente4 termino
Utente5 termino
Utente7 termino
Utente6 termino
Utente1 termino
Utente10 termino
Utente1: utilizzi: 250 occupato: 0
Utente2: utilizzi: 250 occupato: 0
Utente3: utilizzi: 250 occupato: 0
Utente4: utilizzi: 250 occupato: 0
Utente5: utilizzi: 250 occupato: 0
Utente6: utilizzi: 250 occupato: 0
Utente7: utilizzi: 250 occupato: 0
Utente8: utilizzi: 250 occupato: 0
Utente9: utilizzi: 250 occupato: 0
Utente10: utilizzi: 250 occupato: 0
Riepilogo utilizzi
Totale utenti: 2500
Totale Bagno: 2500
Riepilogo occupazione
Totale utenti: 0
Totale Ba

Il programma ora però viene eseguito in modo lento perché la verifica di un bagno libero, la sua occupazione e la sua liberazione vengono eseguiti in modo sincrono. Di conseguenza non è possibile avere un *miss* nel caso di un bagno occupato, in quanto il bagno ad ogni esecuzione del metodo `occupa()` sarà libero.
Questa soluzione non è ottimale, ma è la migliore che possiamo eseguire senza modificare il contenuto del metodo.

#### Synchronized blocks
Sfruttando il metodo dei `synchronized` blocks possiamo rendere più efficiente l'implementazione precedente, mantenendo il programma thread safe.

In [20]:
class BagnoPubblico {
	private int totUtilizzi;
	private int totOccupati;
	private final int disponibili;
	private int occupati;

	public BagnoPubblico(int numeroBagno) {
		this.disponibili = numeroBagno;
		this.occupati = 0;
		this.totUtilizzi = 0;
		this.totOccupati = 0;
	}

	public boolean occupa() {
		// Verifica disponibilita bagni liberi!
		synchronized (this) {
			if (occupati < disponibili) {
				// Bagno libero! Occupa
				occupati++;
				totUtilizzi++;
			} else {
				// Tutti i bagni sono occupati!
				totOccupati++;
				return false;
			}
		}

		// Utilizza il bagno
		utilizzaBagno();

		// Libera il bagno
		synchronized (this){
			occupati--;
		}
		return true;
	}

	private void utilizzaBagno() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(5, 15));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	public int getTotaleUtilizzo() {
		return totUtilizzi;
	}

	public int getTotaleOccupato() {
		return totOccupati;
	}
}

class Utente implements Runnable {
	private final int ID;
	private final BagnoPubblico bp;
	private int numUtilizzi;
	private int numOccupato;

	public Utente(BagnoPubblico bagno, int id) {
		this.bp = bagno;
		this.ID = id;
		this.numUtilizzi = 0;
		this.numOccupato = 0;
	}

	@Override
	public void run() {
		System.out.println(this + " inizio");
		for (int i = 0; i < 250; i++) {
			if (bp.occupa()) {
				numUtilizzi++;
			}
			else {
				numOccupato++;
			}

			// Simula il tempo prima di dover tornare al bagno
			try {
				Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
			} catch (InterruptedException e) {
				e.printStackTrace();
			}
		}
		System.out.println(this + " termino");
	}

	public int getNumUtilizzi() {
		return numUtilizzi;
	}

	public int getNumOccupato() {
		return numOccupato;
	}

	@Override
	public String toString() {
		return "Utente" + ID;
	}
}

In [21]:
BagnoPubblico bp = new BagnoPubblico(2);
List<Thread> allPersons = new ArrayList<>();
List<Utente> allUsers = new ArrayList<>();
for (int i = 1; i <= 10; i++) {
    final Utente user = new Utente(bp, i);
    allUsers.add(user);
    allPersons.add(new Thread(user));
    System.out.println("Creato utente: " + user);
}

allPersons.forEach(Thread::start);

for (Thread thread : allPersons) {
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

int total = 0;
int totalOccupato = 0;
for (Utente utente : allUsers) {
    total += utente.getNumUtilizzi();
    totalOccupato += utente.getNumOccupato();
    System.out.println(
            utente + ": utilizzi: " + utente.getNumUtilizzi() + " occupato: " + utente.getNumOccupato());
}

System.out.println("Riepilogo utilizzi");
System.out.println("Totale utenti: " + total);
System.out.println("Totale Bagno: " + bp.getTotaleUtilizzo());

System.out.println("Riepilogo occupazione");
System.out.println("Totale utenti: " + totalOccupato);
System.out.println("Totale Bagno: " + bp.getTotaleOccupato());

Creato utente: Utente1
Creato utente: Utente2
Creato utente: Utente3
Creato utente: Utente4
Creato utente: Utente5
Creato utente: Utente6
Creato utente: Utente7
Creato utente: Utente8
Creato utente: Utente9
Creato utente: Utente10
Utente2 inizio
Utente1 inizio
Utente4 inizio
Utente9 inizio
Utente10 inizio
Utente8 inizio
Utente3 inizio
Utente6 inizio
Utente7 inizio
Utente5 inizio
Utente6 termino
Utente10 termino
Utente7 termino
Utente2 termino
Utente8 termino
Utente9 termino
Utente5 termino
Utente1 termino
Utente4 termino
Utente3 termino
Utente1: utilizzi: 20 occupato: 230
Utente2: utilizzi: 16 occupato: 234
Utente3: utilizzi: 26 occupato: 224
Utente4: utilizzi: 19 occupato: 231
Utente5: utilizzi: 17 occupato: 233
Utente6: utilizzi: 16 occupato: 234
Utente7: utilizzi: 17 occupato: 233
Utente8: utilizzi: 16 occupato: 234
Utente9: utilizzi: 16 occupato: 234
Utente10: utilizzi: 18 occupato: 232
Riepilogo utilizzi
Totale utenti: 181
Totale Bagno: 181
Riepilogo occupazione
Totale utenti: 231

L'esecuzione è molto veloce, ci sono dei casi di *miss* ed il totale di utenti è concorde con il totale bagno, sia per gli utilizzi che per le occupazioni.

#### Explicit locks
Un ultimo metodo è quello degli explicit locks, modifichiamo dunque il programma come segue:

In [29]:
import java.util.concurrent.ThreadLocalRandom;
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

class BagnoPubblico {
	private int totUtilizzi;
	private int totOccupati;
	private final int disponibili;
	private int occupati;
	private Lock lock;

	public BagnoPubblico(int numeroBagno) {
		this.disponibili = numeroBagno;
		this.occupati = 0;
		this.totUtilizzi = 0;
		this.totOccupati = 0;
		this.lock = new ReentrantLock();
	}

	public Lock getLock() {
		return lock;
	}

	public boolean occupa() {
		// Verifica disponibilita bagni liberi!
		this.getLock().lock();
		if (occupati < disponibili) {
			// Bagno libero! Occupa
			occupati++;
			totUtilizzi++;
		} else {
			// Tutti i bagni sono occupati!
			totOccupati++;
			this.getLock().unlock();
			return false;
		}
		this.getLock().unlock();

		// Utilizza il bagno
		utilizzaBagno();

		// Libera il bagno
		this.getLock().lock();
		occupati--;
		this.getLock().unlock();
		return true;
	}

	private void utilizzaBagno() {
		try {
			Thread.sleep(ThreadLocalRandom.current().nextLong(5, 15));
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}

	public int getTotaleUtilizzo() {
		return totUtilizzi;
	}

	public int getTotaleOccupato() {
		return totOccupati;
	}
}

class Utente implements Runnable {
	private final int ID;
	private final BagnoPubblico bp;
	private int numUtilizzi;
	private int numOccupato;

	public Utente(BagnoPubblico bagno, int id) {
		this.bp = bagno;
		this.ID = id;
		this.numUtilizzi = 0;
		this.numOccupato = 0;
	}

	@Override
	public void run() {
		System.out.println(this + " inizio");
		for (int i = 0; i < 250; i++) {
			if (bp.occupa()) {
				numUtilizzi++;
			}
			else {
				numOccupato++;
			}

			// Simula il tempo prima di dover tornare al bagno
			try {
				Thread.sleep(ThreadLocalRandom.current().nextLong(1, 5));
			} catch (InterruptedException e) {
				e.printStackTrace();
			}
		}
		System.out.println(this + " termino");
	}

	public int getNumUtilizzi() {
		return numUtilizzi;
	}

	public int getNumOccupato() {
		return numOccupato;
	}

	@Override
	public String toString() {
		return "Utente" + ID;
	}
}

In [30]:
BagnoPubblico bp = new BagnoPubblico(2);

List<Thread> allPersons = new ArrayList<>();
List<Utente> allUsers = new ArrayList<>();
for (int i = 1; i <= 10; i++) {
    final Utente user = new Utente(bp, i);
    allUsers.add(user);
    allPersons.add(new Thread(user));
    System.out.println("Creato utente: " + user);
}

allPersons.forEach(Thread::start);

for (Thread thread : allPersons) {
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

int total = 0;
int totalOccupato = 0;
for (Utente utente : allUsers) {
    total += utente.getNumUtilizzi();
    totalOccupato += utente.getNumOccupato();
    System.out.println(
            utente + ": utilizzi: " + utente.getNumUtilizzi() + " occupato: " + utente.getNumOccupato());
}

System.out.println("Riepilogo utilizzi");
System.out.println("Totale utenti: " + total);
System.out.println("Totale Bagno: " + bp.getTotaleUtilizzo());

System.out.println("Riepilogo occupazione");
System.out.println("Totale utenti: " + totalOccupato);
System.out.println("Totale Bagno: " + bp.getTotaleOccupato());

Creato utente: Utente1
Creato utente: Utente2
Creato utente: Utente3
Creato utente: Utente4
Creato utente: Utente5
Creato utente: Utente6
Creato utente: Utente7
Creato utente: Utente8
Creato utente: Utente9
Creato utente: Utente10
Utente4 inizio
Utente5 inizio
Utente9 inizio
Utente7 inizio
Utente3 inizio
Utente2 inizio
Utente10 inizio
Utente1 inizio
Utente8 inizio
Utente6 inizio
Utente10 termino
Utente8 termino
Utente4 termino
Utente2 termino
Utente1 termino
Utente3 termino
Utente6 termino
Utente5 termino
Utente7 termino
Utente9 termino
Utente1: utilizzi: 17 occupato: 233
Utente2: utilizzi: 14 occupato: 236
Utente3: utilizzi: 16 occupato: 234
Utente4: utilizzi: 14 occupato: 236
Utente5: utilizzi: 25 occupato: 225
Utente6: utilizzi: 19 occupato: 231
Utente7: utilizzi: 29 occupato: 221
Utente8: utilizzi: 10 occupato: 240
Utente9: utilizzi: 26 occupato: 224
Utente10: utilizzi: 11 occupato: 239
Riepilogo utilizzi
Totale utenti: 181
Totale Bagno: 181
Riepilogo occupazione
Totale utenti: 231

Anche in questo caso, l'esecuzione è multi-thread e sopratutto thread-safe.